In [ ]:
#%%

from typing import NamedTuple, List
import random
from random import uniform
import matplotlib.pyplot as plt
import math

# 2D world/earth surface
# one "world coordinate frame", origin (0, 0), x-axis to the right, y-axis up

Meter = float
Radian = float

class Point(NamedTuple):
    x: Meter
    y: Meter
    
class Circle(NamedTuple):
    p: Point
    r: Meter #radius

class Star(NamedTuple):
    p: Point
    # brightness: float

class Camera(NamedTuple):
    """
    Assume camera is pointing directly away from earth, so the camera
    pose is representing by one number, it's position on the surface of 
    the earth.  position == angle in [-pi, pi)
    maybe use [0, 2pi]?
    """
    angle: float  # "position" of camera on the earth
    fov: Degree   # full cone angle, assume centered on center of image array
    npixels: int  # number of pixels in final image array
        
Image = List[int]  # one row of pixels for our toy 2D world

Here, `generate_starfield` takes a `margin` that makes sure stars are not inside of the earth by a fixed `margin`. `generate_starfield` also takes `limit` to measure the boundaries of the outmost universe.

In [ ]:
"""
def cam_xy(cam: Camera) -> Point:
    pass
"""

def distance_point_circle(p: Point, earth: Circle) -> Meter:
    # d = distance between p and center of circle
    d = math.sqrt((p.x - earth.p.x) ** 2 + (p.y - earth.p.y) ** 2)
    return d #returns distance

def out_earth(margin,p:Point, earth:Circle) -> bool:
    # distance from point to circle is less than or equal to the radius of the earth
    # .     (      .      )
    #margin = 1.0
    return distance_point_circle(p, earth) >= earth.r + margin

def in_earth(margin,p:Point, earth:Circle) -> bool:
    # distance from point to circle is less than or equal to the radius of the earth
    # .     (      .      )
    #margin = 1.0
    return distance_point_circle(p, earth) + margin <= earth.r 
# random star field
"""
arguments: n_stars, circle struct to pull radius from and to get boundaries, margin, and universe size
"""
def generate_starfield(margin,earth:Circle,limits:Point,n_stars=100) -> List[Star]:
  y = []
  x = []
  counter = 0
  while (counter < n_stars):
    random_xs = uniform(-1*limits.x,limits.x)
    random_ys = uniform(-1*limits.y,limits.y)
    k =  out_earth(margin,Point(random_xs,random_ys), earth )
    #print("Are the stars outside of Earth and the margin?" , k)
    if (k == True):
      y.append(random_ys)
      x.append(random_xs)
      counter +=1
    else:
      counter +=0
  
  return y, x

def generate_camera(margin, earth:Circle):
  x = True
  while (x == True):
    random_x = uniform(-1*earth.r,earth.r)
    random_y = uniform(-1*earth.r,earth.r)

    k = in_earth(margin,Point(random_x,random_y),earth)
    #print("Is camera within earth and the margin? " ,k)
    if (k == True):
      x = False
  return Point(random_x,random_y)

  

In [ ]:
def getimage(cam: Camera, starfield: List[Star]) -> Image:
    image = [0] * cam.npixels
    # 1. project each star onto "camera imager/detector/film"
    for star in starfield:
        if abs(get_relative_heading(cam, star)) < cam.fov / 2.0:
            # transform star world coords to camera frame
            X, Y = ...
            # perspective projection
            x = f * X / Y
            # light up the appropriate pixel, i, corresponding to x
            i = ...
    return image

In [ ]:
"""
Consider making a function to generate the matplotlib
"""
#int main(){}
def main():
  #creating earth and stars
  earth  = Circle(Point(0.0, 0.0), 10) #default coords should always be zero
  limits = Point(30,30)
  y,x    = generate_starfield(0,earth,limits,100)
  # draw earth and stars and camera
  circle1 = plt.Circle((earth.p.x, earth.p.y), earth.r , color='b',fill=False)
  camera  = generate_camera(5, earth)
  #Configuring settings to center the circle onto the origin and zoom out
  plt.style.use('ggplot')
  fig, ax = plt.subplots()
  ax.spines['left'].set_position('zero')
  ax.spines['right'].set_color('none')
  ax.spines['bottom'].set_position('zero')
  ax.spines['top'].set_color('none')

  lims = (-40,40)
  plt.xlim(lims)
  plt.ylim(lims)

  ax.add_artist(circle1)
  plt.scatter(x,y)
  plt.scatter(camera.x, camera.y, marker='^', c =2)
  plt.plot()
  plt.show()
  # simulated image

  # star map

  # cross correlation image matching to known map

if __name__ == "__main__":
  main()
